In [ ]:
import pandas as pd
import numpy as np

df_costumers = pd.read_csv("olist_customers_dataset.csv")
df_orders = pd.read_csv("olist_orders_dataset.csv")
df_items = pd.read_csv("olist_order_items_dataset.csv")
df_products = pd.read_csv("olist_products_dataset.csv")
df_reviews = pd.read_csv("olist_order_reviews_dataset.csv")
df_gelocations = pd.read_csv("olist_geolocation_dataset.csv")